<a href="https://colab.research.google.com/github/Kauajr13/Unesp-bcc/blob/main/IA/trabalho_agentes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Trabalho: **Meu Primeiro Agente** — Aspirador (2 salas)

**Aluno(a):** Kauã Junior Silva Soares

**Disciplina:** Inteligência Artificial  

**Data:** 15/09/2025

> **Importante:** Este notebook contém **esqueletos** com parte da solução já implementada.
> Você deve implementar as demais funções pedidas, rodar experimentos e responder às perguntas.



## Objetivos
1. Definir o problema via **PEAS**.  
2. Especificar formalmente **S, A, T, h, r, P** e explicar cada símbolo.  
3. Implementar um **agente reflexo** e medir a **performance** passo a passo.  
4. Discutir **limitações** e propor **uma melhoria simples**.



## Instruções
- Preencha todo conteúdo solicitado (texto e código).  
- Use a notação apresentada na aula: `s=(p,d_L,d_R)`, `o=h(s)`, `a∈A`, `r(s,a,s')`, `P`.  
- Quando terminar, exporte em **PDF** e entregue **PDF + .ipynb** no _classroom_.



---
## 1) PEAS — Definição do problema
Explique concisamente e escreva as fórmulas pedidas.

**P — Performance:**  
- Defina `r(s,a,s')` e $P = \sum_{t=0}^{H-1} r_t$
  - `r(s,a,s')` é a recompensa recebida após a ação `a` levar do estado `s` ao estado `s′`
  - `$P = \sum_{t=0}^{H-1} r_t$` seria o desempenho total ao longo de H passos, sendo dessa forma a soma de todas as recompensas instantâneas

- Declare os parâmetros (λ e custos).
  - λ>0: ganho (valor) por remoção de uma unidade de sujeira
  - os custos seriam os custos por ação realizada, no aspirador seria o custo de ir para esquerda,direita ou aspirar

**E — Environment:**  
- Descreva as duas salas, estados e por que |S| = 8.
  - O estado s é s=(p,dL,dR) onde:
    - p ∈ {L,R} é a posição do agente (esquerda ou direita);
    - dL ∈ {0,1} é o bit de sujeira da sala esquerda (0=limpa, 1=suja);
    - dR ∈ {0,1} é o bit de sujeira da sala direita (0=limpa, 1=suja);
    - Como cada componente apresenta 2 escolhas possíveis, logo o número de estados é |S| = 2^3 = 8

**A — Actuators:**  
- Liste {ESQ, DIR, ASP} e comente custos.
  - ESQ: move o agente para a sala da esquerda
  - DIR: move o agente para a sala da direita
  - ASP: aspira a sala atual, removendo a sujeira se houver

  - custos:
    - c(ESQ): custo de se mover para a esquerda
    - c(DIR): custo de se mover para a direita
    - c(ASP): custo de aspirar


**S — Sensors:**  
- Defina `h: S -> O` no modo local (`h(s)=(p, sujeira_aqui)`).

  - Função de percepção: `h: S -> O` definida no modo local por `h(s)=(p, sujeira_aqui)` onde `p` é a posição do agente e `sujeira_aqui` é o bit de sujeira onde o agente está


---
## 2) Especificação formal — S, A, T, h, r, P  **(TODO)**
Preencha a definição com sua própria redação e notação clara.

- **Estados:**  `S={(p,dL​,dR​)∣p∈{L,R},dL​∈{0,1},dR​∈{0,1}}`  
- **Ações:**    `A = {ESQ, DIR, ASP}`  
- **Transição determinística:** `T: S x A -> S`  
  - `T(s,ASP) = {(L,0,dR​)}, se p=L, ou {(R,dL​,0)}, se p=R`
  - `T(s,DIR) = (R,dL​,dR​)`
  - `T(s,ESQ) = (L,dL,dR)`
- **Percepção:** `h: S -> O` com observação local
    - `h(p,dL​,dR​)=(p,dp​)`
    - `onde dp=dL se p=L e dP = dR se p = R`
- **Recompensa r e performance P:**
  - λ>0: valor ganho por remover uma unidade de sujeira
  - c(ESQ), c(DIR), c(ASP) >= 0 : custos das ações
  - H : horizonte finito do episódio
  - Recompensa imediata: `r(s, a, s') = λ * 1{ a = ASP  and  d_p(s) = 1  and  d_p(s') = 0 }  -  c(a)`

  - Performance: `P = sum_{t=0}^{H-1} r_t onde r_t = r(s_t, a_t, s_{t+1})`

In [5]:
# inicio do código

from dataclasses import dataclass
from typing import Tuple, List, Dict, Any

import pandas as pd


In [6]:
@dataclass
class Env2Salas:
    p: str   # 'L' ou 'R'
    dL: int  # 0=limpo, 1=sujo
    dR: int  # 0=limpo, 1=sujo

    def copy(self):
        return Env2Salas(self.p, self.dL, self.dR)

    def h(self) -> Tuple[str, int]:
        """Retorna (posicao, sujeira_aqui)."""
        sujeira_aqui = self.dL if self.p == 'L' else self.dR
        return (self.p, sujeira_aqui)

    def T(self, a: str) -> 'Env2Salas':
        """Transição determinística."""

        s_ = self.copy()

        if a == 'ASP':
            if s_.p == 'L':
                s_.dL = 0
            else:
                s_.dR = 0
        elif a == 'DIR':
            s_.p = 'R'
        elif a == 'ESQ':
            s_.p = 'L'

        return s_

In [7]:
def r(s: Env2Salas, a: str, s_: Env2Salas, lam: float,
      c_esq: float, c_dir: float, c_asp: float) -> float:
    """Recompensa imediata."""
    custos = {'ESQ': c_esq, 'DIR': c_dir, 'ASP': c_asp}
    custo_a = custos.get(a, 0) # Retorna 0 para ações desconhecidas

    recompensa_limpeza = 0.0
    if a == 'ASP':
        if s.p == 'L' and s.dL == 1 and s_.dL == 0:
            recompensa_limpeza = lam
        elif s.p == 'R' and s.dR == 1 and s_.dR == 0:
            recompensa_limpeza = lam

    return recompensa_limpeza - custo_a

In [8]:
def politica_reflexo(obs: Tuple[str, int]) -> str:
    """Política reflexo simples."""
    p, sujeira_aqui = obs
    if sujeira_aqui == 1:
        return 'ASP'
    else:
        return 'DIR' if p == 'L' else 'ESQ'

In [9]:
def simular(s0: Env2Salas, lam=1.0, c_esq=1.0, c_dir=1.0, c_asp=1.0, H=20):
    """Roda a simulação por H passos ou até o ambiente estar totalmente limpo."""
    historico = []
    s = s0.copy()

    for t in range(H):
        if s.dL == 0 and s.dR == 0:
            break

        obs = s.h()
        a = politica_reflexo(obs)
        s_ = s.T(a)

        recompensa_imediata = r(s, a, s_, lam, c_esq, c_dir, c_asp)

        historico.append({
            't': t,
            's': s.copy(),
            'a': a,
            's_': s_.copy(),
            'r': recompensa_imediata
        })

        s = s_

    df_historico = pd.DataFrame(historico)

    recompensa_total = df_historico['r'].sum()

    return historico, recompensa_total


### Testes mínimos (rode quando terminar as implementações)
Descomente os `assert` abaixo **depois** de implementar `h`, `T`, `r`, `politica_reflexo`, `simular`.


In [10]:

# # --- Descomente os asserts após implementar ---
s = Env2Salas('L',1,1)
assert s.h() == ('L', 1)
s1 = s.T('ASP')
assert (s1.p, s1.dL, s1.dR) == ('L', 0, 1)
s2 = s1.T('DIR')
assert (s2.p, s2.dL, s2.dR) == ('R', 0, 1)

# Recompensas (λ=1; todos custos=1)
assert abs(r(s, 'ASP', s1, lam=1, c_esq=1, c_dir=1, c_asp=1) - 0.0) < 1e-9
assert abs(r(s1, 'DIR', s2, lam=1, c_esq=1, c_dir=1, c_asp=1) + 1.0) < 1e-9

# Política reflexo
assert politica_reflexo(('L',1)) == 'ASP'
assert politica_reflexo(('L',0)) == 'DIR'
assert politica_reflexo(('R',0)) == 'ESQ'

# Simulação
hist, P = simular(Env2Salas('L',1,1), lam=1, c_esq=1, c_dir=1, c_asp=1, H=20)
assert isinstance(hist, list) and len(hist) >= 1


---
## 4) Experimentos e Performance

### Cenário A (base)
Parâmetros: `λ=1`, `c(ESQ)=c(DIR)=c(ASP)=1`, `s0=(L,1,1)`.

- Rode a simulação e mostre uma **tabela** com as colunas: `t, s, a, s_, r`.
- Calcule o **P** final e comente o resultado (2–4 linhas).


In [11]:

# TODO: rode o Cenário A e exiba a tabela e o P
s0 = Env2Salas('L', 1, 1)
hist_A, P_A = simular(s0, lam=1, c_esq=1, c_dir=1, c_asp=1, H=20)
df_A = pd.DataFrame(hist_A)

print("Tabela da simulação (Cenário A):")
print(df_A)

print(f"\nRecompensa total (P) para o Cenário A = {P_A}")

#   Tabela da simulação (Cenário A):
#      t                             s    a                            s_    r
#   0  0  Env2Salas(p='L', dL=1, dR=1)  ASP  Env2Salas(p='L', dL=0, dR=1)  0.0
#   1  1  Env2Salas(p='L', dL=0, dR=1)  DIR  Env2Salas(p='R', dL=0, dR=1) -1.0
#   2  2  Env2Salas(p='R', dL=0, dR=1)  ASP  Env2Salas(p='R', dL=0, dR=0)  0.0

#   Recompensa total (P) para o Cenário A = -1.0


Tabela da simulação (Cenário A):
   t                             s    a                            s_    r
0  0  Env2Salas(p='L', dL=1, dR=1)  ASP  Env2Salas(p='L', dL=0, dR=1)  0.0
1  1  Env2Salas(p='L', dL=0, dR=1)  DIR  Env2Salas(p='R', dL=0, dR=1) -1.0
2  2  Env2Salas(p='R', dL=0, dR=1)  ASP  Env2Salas(p='R', dL=0, dR=0)  0.0

Recompensa total (P) para o Cenário A = -1.0



### Cenário B (limpar vale mais)
Parâmetros: `λ=2`, custos = 1. Rode e compare com o Cenário A (2–4 linhas).


In [12]:

# TODO: Cenário B
s0 = Env2Salas('L',1,1)
hist_B, P_B = simular(s0, lam=2, c_esq=1, c_dir=1, c_asp=1, H=20)
pd.DataFrame(hist_B)
print("P (Cenário B) =", P_B)

# P (Cenário B) = 1.0


P (Cenário B) = 1.0



### Cenário C (movimento caro)
Parâmetros: `λ=1`, `c(ESQ)=c(DIR)=2`, `c(ASP)=1`. Discuta o impacto (2–4 linhas).


In [13]:

# TODO: Cenário C
s0 = Env2Salas('L',1,1)
hist_C, P_C = simular(s0, lam=1, c_esq=2, c_dir=2, c_asp=1, H=20)
pd.DataFrame(hist_C)
print("P (Cenário C) =", P_C)

# P (Cenário C) = -2.0


P (Cenário C) = -2.0



---
## 5) Discussão e melhoria da política
- Explique **duas limitações** do agente reflexo (sem memória).  
  - Não consegue lidar bem com informação incompleta: se só enxerga a sujeira da sala atual, não sabe se a outra precisa de limpeza e pode acabar preso limpando sempre o mesmo lugar.
  - Também pode agir de forma repetitiva, visto que, sem memória, repete decisões e pode ficar indo e voltando entre salas limpas, gastando energia à toa.
- Proponha **uma melhoria simples** (ex.: 1 bit de memória) e **implemente** abaixo.

  - Melhoria usando 1 bit de memória

In [ ]:
def politica_melhorada(obs: Tuple[str, int], memoria: List[int]) -> str:
    """Política aprimorada com um bit de memória."""
    p, sujeira_aqui = obs

    # Se a sala atual está suja, a ação é aspirar.
    if sujeira_aqui == 1:
        return 'ASP'

    # Se a sala atual está limpa, a decisão depende da memória.
    else:
        # Se a outra sala ainda não foi visitada ou verificada
        if memoria[0] == 0:
            memoria[0] = 1 # Marca que visitou esta sala
            return 'DIR' if p == 'L' else 'ESQ'
        else:
            # Ambas as salas já foram limpas
            return 'PARAR' # Nova ação para parar o movimento

def simular_melhorado(s0: Env2Salas, lam=1.0, c_esq=1.0, c_dir=1.0, c_asp=1.0, H=20):
    historico = []
    s = s0.copy()
    memoria = [0] # 0 = não visitou a outra sala, 1 = visitou

    for t in range(H):
        if s.dL == 0 and s.dR == 0 and memoria[0] == 1:
            break

        obs = s.h()
        a = politica_reflexo_com_memoria(obs, memoria)

        # Implementa a nova ação 'PARAR'
        if a == 'PARAR':
            historico.append({'t': t, 's': s.copy(), 'a': a, 's_': s.copy(), 'r': 0.0})
            break

        s_ = s.T(a)
        recompensa_imediata = r(s, a, s_, lam, c_esq, c_dir, c_asp)
        historico.append({'t': t, 's': s.copy(), 'a': a, 's_': s_.copy(), 'r': recompensa_imediata})
        s = s_

    df_historico = pd.DataFrame(historico)
    recompensa_total = df_historico['r'].sum()

    return historico, recompensa_total



---
## 6) Bônus (opcional)
- **3 salas (L–C–R):** defina S, A, T, h e rode um cenário; calcule |S|.  
- **Ruído de sujeira (prob. ρ):** estenda a transição e estime E[P] (média de 50 runs).
